In [51]:
client_id = "86ac0958ab204b55bf2159768e863ea8"
client_secret = "e99322bd21d2460a94f3401976022220"

In [52]:
import spotipy
import requests
import matplotlib 
from spotipy.oauth2 import SpotifyOAuth
import sqlite3
import time 
import requests 
from datetime import datetime

In [53]:
pip install spotipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [54]:


scope = 'user-read-recently-played'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri = "http://localhost:8888/callback", scope=scope, show_dialog = True ))



In [55]:
#limit = number of tracks it will return, after = is a timestamp in miliseconds(basically songs played '9-5' but we give the time in unix), before = same but before. CANNOT USE BEFORE AND AFTER AT THE SAME TIME
recent = sp.current_user_recently_played(limit=5, after=None, before=None)


In [63]:
recent

{'items': [{'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7vWlb4pM85jCHvV771qZZW'},
       'href': 'https://api.spotify.com/v1/artists/7vWlb4pM85jCHvV771qZZW',
       'id': '7vWlb4pM85jCHvV771qZZW',
       'name': 'Tony! Toni! Toné!',
       'type': 'artist',
       'uri': 'spotify:artist:7vWlb4pM85jCHvV771qZZW'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
      'MY',
      'MT',
      'MX',
      'NL',
      'NZ',
      'NI',
      'NO',
      'PA',
      'PY',
      'PE',
      'PH',
      'PL',
      'PT',
      'SG',
      'SK',
      'ES',
      'SE

In [57]:
# Connect to SQLite database
conn = sqlite3.connect('music_weather.db')
cur = conn.cursor()


In [58]:
# Create tables
cur.execute('''
CREATE TABLE IF NOT EXISTS SpotifyTracks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    track_name TEXT,
    artist TEXT,
    album TEXT,
    played_at INTEGER,
    duration_ms INTEGER
)
''')
cur.execute('''
CREATE TABLE IF NOT EXISTS WeatherData (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT,
    hour TEXT,
    temp_c REAL,
    condition TEXT,
    wind_mph REAL,
    humidity INTEGER,
    location TEXT
)
''')

conn.commit()



In [59]:
# Function to fetch and store recently played tracks
def fetch_spotify_tracks():
    recent_tracks = sp.current_user_recently_played(limit=25)
    for item in recent_tracks['items']:
        track = item['track']
        track_name = track['name']
        artist = track['artists'][0]['name']
        album = track['album']['name']
        played_at = int(time.mktime(time.strptime(item['played_at'][:19], "%Y-%m-%dT%H:%M:%S")))
        duration_ms = track['duration_ms']

        # Insert into database
        cur.execute('''
        INSERT INTO SpotifyTracks (track_name, artist, album, played_at, duration_ms)
        VALUES (?, ?, ?, ?, ?)
        ''', (track_name, artist, album, played_at, duration_ms))

    conn.commit()

# Fetch and store Spotify data
fetch_spotify_tracks()

In [71]:
#This cowork request the weather for url
AP_key = "e2566028033342cebf212759240312"
location = ["Ann Arbor", "Grand Rapids", "Detroit", "Traverse City"]           
days = 7
for locations in location:

    url = f"http://api.weatherapi.com/v1/forecast.json?key={AP_key}&q={locations}&days={days}"
    response = requests.get(url)
    
    if response.status_code == 200:
            print(f"Fetching data for {location}")
            data = response.json()

            # Check the structure of the data
            print(f"Sample data for {location}: {data['forecast']['forecastday'][0]}")

            for day in data["forecast"]["forecastday"]:
                for hour_data in day["hour"]:
                    date = hour_data["time"][:10]
                    hour = hour_data["time"][11:]
                    temp_c = hour_data["temp_c"]
                    condition = hour_data["condition"]["text"]
                    wind_mph = hour_data["wind_mph"]
                    humidity = hour_data["humidity"]

                    # Insert into database
                    cur.execute('''
                    INSERT INTO WeatherData (date, hour, temp_c, condition, wind_mph, humidity)
                    VALUES (?, ?, ?, ?, ?, ?)
                    ''', (date, hour, temp_c, condition, wind_mph, humidity))

            conn.commit()
    else:
            print(f"Failed to fetch data for {location}. Status Code: {response.status_code}")

Fetching data for ['Ann Arbor', 'Grand Rapids', 'Detroit', 'Traverse City']
Sample data for ['Ann Arbor', 'Grand Rapids', 'Detroit', 'Traverse City']: {'date': '2024-12-05', 'date_epoch': 1733356800, 'day': {'maxtemp_c': -0.7, 'maxtemp_f': 30.7, 'mintemp_c': -7.2, 'mintemp_f': 19.0, 'avgtemp_c': -3.9, 'avgtemp_f': 25.0, 'maxwind_mph': 25.3, 'maxwind_kph': 40.7, 'totalprecip_mm': 1.6, 'totalprecip_in': 0.06, 'totalsnow_cm': 0.0, 'avgvis_km': 6.5, 'avgvis_miles': 4.0, 'avghumidity': 71, 'daily_will_it_rain': 1, 'daily_chance_of_rain': 89, 'daily_will_it_snow': 0, 'daily_chance_of_snow': 3, 'condition': {'text': 'Moderate or heavy snow showers', 'icon': '//cdn.weatherapi.com/weather/64x64/day/371.png', 'code': 1258}, 'uv': 0.3}, 'astro': {'sunrise': '07:49 AM', 'sunset': '05:03 PM', 'moonrise': '11:47 AM', 'moonset': '09:15 PM', 'moon_phase': 'Waxing Crescent', 'moon_illumination': 14, 'is_moon_up': 0, 'is_sun_up': 0}, 'hour': [{'time_epoch': 1733374800, 'time': '2024-12-05 00:00', 'temp_

In [72]:
data["forecast"]["forecastday"][0]["hour"]

[{'time_epoch': 1733374800,
  'time': '2024-12-05 00:00',
  'temp_c': -4.3,
  'temp_f': 24.3,
  'is_day': 0,
  'condition': {'text': 'Overcast ',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png',
   'code': 1009},
  'wind_mph': 15.9,
  'wind_kph': 25.6,
  'wind_degree': 311,
  'wind_dir': 'NW',
  'pressure_mb': 1002.0,
  'pressure_in': 29.59,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'snow_cm': 0.0,
  'humidity': 87,
  'cloud': 100,
  'feelslike_c': -11.9,
  'feelslike_f': 10.6,
  'windchill_c': -11.9,
  'windchill_f': 10.6,
  'heatindex_c': -4.3,
  'heatindex_f': 24.3,
  'dewpoint_c': -6.6,
  'dewpoint_f': 20.2,
  'will_it_rain': 0,
  'chance_of_rain': 0,
  'will_it_snow': 0,
  'chance_of_snow': 0,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'gust_mph': 21.0,
  'gust_kph': 33.9,
  'uv': 0},
 {'time_epoch': 1733378400,
  'time': '2024-12-05 01:00',
  'temp_c': -4.3,
  'temp_f': 24.3,
  'is_day': 0,
  'condition': {'text': 'Overcast ',
   'icon': '//cdn.weatherapi.com/weather/6

In [73]:
def fetch_weather_data():
  

    for day in data["forecast"]["forecastday"]:
                for hour_data in day["hour"]:
                    date = hour_data["time"][:9]
                    hour = hour_data["time"][9:]
                    temp_c = hour_data["temp_c"]
                    condition = hour_data["condition"]["text"]
                    wind_mph = hour_data["wind_mph"]
                    humidity = hour_data["humidity"]

                # Insert into database
                    cur.execute('''
                    INSERT INTO WeatherData (date, hour, temp_c, condition, wind_mph, humidity)
                    VALUES (?, ?, ?, ?, ?, ?)
                    ''', (date, hour, temp_c, condition, wind_mph, humidity))

    conn.commit()
    


In [74]:
# This function is useful for quickly checking the integrity 
#and basic structure of the WeatherData table.
def verify_weather_data():
    cur.execute("SELECT COUNT(*) FROM WeatherData")
    total_rows = cur.fetchone()[0]
    print(f"Total weather data rows: {total_rows}")

    cur.execute("SELECT * FROM WeatherData LIMIT 5")
    sample_rows = cur.fetchall()
    print("Sample weather data:", sample_rows)

# Call the function to verify data
verify_weather_data()


Total weather data rows: 672
Sample weather data: [(1, '2024-12-05', '00:00', -2.1, 'Overcast ', 21.7, 93, None), (2, '2024-12-05', '01:00', -3.1, 'Cloudy ', 23.5, 83, None), (3, '2024-12-05', '02:00', -4.1, 'Blowing snow', 22.6, 87, None), (4, '2024-12-05', '03:00', -5.1, 'Overcast ', 23.7, 80, None), (5, '2024-12-05', '04:00', -5.4, 'Partly Cloudy ', 22.8, 79, None)]


In [77]:
def check_duplicates():
    query = '''
    SELECT date, hour, COUNT(*)
    FROM WeatherData
    GROUP BY date, hour
    HAVING COUNT(*) > 1
    '''
    cur.execute(query)
    duplicates = cur.fetchall()
    if duplicates:
        print(f"Found duplicates: {duplicates}")
    else:
        print("No duplicates found.")

# Call the function
check_duplicates()


No duplicates found.


In [76]:
def remove_duplicates():
    query = '''
    DELETE FROM WeatherData
    WHERE ROWID NOT IN (
        SELECT MIN(ROWID)
        FROM WeatherData
        GROUP BY date, hour, location
    )
    '''
    cur.execute(query)
    conn.commit()
    print("Duplicates removed successfully.")

# Call the function
remove_duplicates()


Duplicates removed successfully.


In [80]:

def 
find_duplicate_spotify_tracks():
    # Define an SQL query to remove duplicate records from the SpotifyTracks table.
    # Duplicates are identified based on having the same track_name, artist, and album.
    # ROWID is used to ensure only one record of each duplicate group is kept.
    query = '''
    SELECT track_name, artist, album, COUNT(*) 
    FROM SpotifyTracks
    GROUP BY track_name, artist, album
    HAVING COUNT(*) > 1
    '''
    cur.execute(query)
    duplicates = cur.fetchall()

    if duplicates:
        print(f"Found duplicates: {duplicates}")
    else:
        print("No duplicates found.")

# Call the function
find_duplicate_spotify_tracks()


No duplicates found.


In [79]:
def remove_duplicate_spotify_tracks():
    query = '''
    DELETE FROM SpotifyTracks
    WHERE ROWID NOT IN (
        SELECT MIN(ROWID)
        FROM SpotifyTracks
        GROUP BY track_name, artist, album
    )
    '''
    cur.execute(query)
    conn.commit()
    print("Duplicates removed successfully.")

# Call the function
remove_duplicate_spotify_tracks()


Duplicates removed successfully.
